In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras import  models, optimizers, layers, activations
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D ,GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6958962659163926141
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2258055988
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9602183712390426133
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
path = "E:\deep_learning_project\inaturalist_12K"

In [5]:
image_size=(256,256,3)
output_size=10

In [6]:
def get_data(path,augmentation=True):
    train_path=os.path.join(path,"train")
    test_path=os.path.join(path,"val")
    if augmentation==False:
        train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
    else:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_data = train_generator.flow_from_directory(
                                                        directory=train_path,
                                                        target_size=(256, 256),
                                                        color_mode="rgb",
                                                        batch_size=32,
                                                        class_mode="categorical",
                                                        shuffle=True,
                                                        seed=42)
    valid_data=train_generator.flow_from_directory(
                                                        directory=train_path,
                                                        target_size=(256, 256),
                                                        color_mode="rgb",
                                                        batch_size=32,
                                                        class_mode="categorical",
                                                        shuffle=True,
                                                        seed=42)
    test_data=test_generator.flow_from_directory(
                                                        directory=test_path,
                                                        target_size=(256, 256),
                                                        color_mode="rgb",
                                                        batch_size=32,
                                                        class_mode="categorical",
                                                        shuffle=True,
                                                        seed=42)
    return train_data,valid_data,test_data

In [7]:
model_b=False

In [8]:
class inc_v2(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=InceptionResNetV2(include_top=False,
            weights="imagenet",
            input_shape=input_size,
        )
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))



In [9]:
class inc_v3(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=InceptionV3(include_top=False,
            weights="imagenet", 
            input_shape=input_size,
        )
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))

In [10]:
class Xcpt_ion(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=Xception(include_top=False,
            weights="imagenet",
            input_shape=input_size)
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))

In [11]:
class Res_net(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=ResNet50(include_top=False,
            weights="imagenet",
            input_shape=input_size)
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))

In [12]:
model=Res_net(image_size,output_size,0.4,True,32)
model.model.summary()

print(model.model_b.trainable)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                4194336   
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [13]:
trained_models={"inception_v2":inc_v2,"inception_v3":inc_v3,"Xception":Xcpt_ion,"Resnet_50":Res_net}

In [14]:
import wandb
wandb.login()

wandb: Currently logged in as: akashsainics21m003 (use `wandb login --relogin` to force relogin)


True

In [15]:
from wandb.keras import WandbCallback

# FINETUNING PRETRAINED MODELS 

In [16]:

import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
class Runner(object):
    def __init__(self,inaturalist_data=True,augmentation=True):
        if inaturalist_data:
            self.initialize_inaturalist_data(augmentation)
        else:
            self.initialize_data()

    def initialize_inaturalist_data(self,augmentation=True):
        self.train_data,self.valid_data,self.test_data= get_data(path,augmentation)
    

    def initialize_data(self):
        raise NotImplementedError("Please implement this method if you need other dataset.")

    @staticmethod
    def get_activation_function(key):
        mapper = {
      #"sigmoid": Sigmoid,
      "relu": 'relu'
      #"tanh": Tanh
    }
        assert key in mapper
        return mapper[key]

def run_wandb():
    wandb.init(project="pretrained_model", entity="cs21m003_cs21d406")
    config = wandb.config
    base_model=trained_models[config.models]
    wandb.run.name=f"model_{config.models}_e_{config.epochs}_bs_{config.batch_size}_rate_{config.learning_rate}_aug_{config.augmentation}_BN_{config.batch_normalization}_drp_{config.drop_out}_dense_{config.dense_size}"
    runner=Runner(True,config.augmentation)
    """
    params = {
    "epochs"        : config.epochs,
    "batch_size"    : config.batch_size,
    "act_func"      : config.act_func,
    "learning_rate" : config.learning_rate,
    "augmentation"  : config.augmentation,
    "batch_normalization"  : config.batch_normalization,
    "drop_out"  : config.drop_out,
    "dense_size":config.dense_size,
    "models": config.models

    }
    """
    model_1=base_model(image_size,output_size,config.drop_out,config.batch_normalization,config.dense_size)
    model_1.model.compile(
    optimizer=Adam(config.learning_rate),  # Optimizer
    loss="categorical_crossentropy", metrics="categorical_accuracy")
    model_1.model.fit(runner.train_data,epochs=config.epochs,batch_size=config.batch_size,validation_data=runner.valid_data,
          callbacks=[WandbCallback()])
    loss,accuracy=model_1.model.evaluate(runner.test_data,batch_size=config.batch_size)
    model_1.model_b.trainable=True
    print(model_1.model_b.trainable)
    print(f'test accuracy:{accuracy}')
    wandb.log({"test accuracy":accuracy})

def do_hyperparameter_search_using_wandb():
    sweep_config = {
    "name": "random sweep",
    "method": "random",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs": {"values": [5,10]}, 
      "batch_size": {"values": [64]}, 
      #"act_func": {"values": ['elu', 'relu', 'selu']}, 
      "learning_rate": {"values": [1e-3, 1e-4]}, 
      "augmentation": {"values": [True,False]} , 
      "batch_normalization": {"values": [True,False]},
        
      "drop_out": {"values": [0.3,0.4,0.5]},
      "models": {"values": ["inception_v2"]},
      "dense_size": {"values": [32,64]}}}
  
    sweep_id = wandb.sweep(sweep_config, project = "pretrained_model",entity='cs21m003_cs21d406')
    wandb.agent(sweep_id, function=run_wandb,count=2)

In [ ]:
do_hyperparameter_search_using_wandb()

Create sweep with ID: 966enmsy
Sweep URL: https://wandb.ai/cs21m003_cs21d406/pretrained_model/sweeps/966enmsy


wandb: Agent Starting Run: 9nay3qo7 with config:
wandb: 	augmentation: False
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	dense_size: 32
wandb: 	drop_out: 0.5
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	models: inception_v2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
313/313 [==============================] - 589s 2s/step - loss: 3.3309 - categorical_accuracy: 0.2149 - val_loss: 1.8364 - val_categorical_accuracy: 0.3243
Epoch 2/5
313/313 [==============================] - 448s 1s/step - loss: 2.5998 - categorical_accuracy: 0.2395 - val_loss: 1.6901 - val_categorical_accuracy: 0.3800
Epoch 3/5
313/313 [==============================] - 446s 1s/step - loss: 2.1716 - categorical_accuracy: 0.2657 - val_loss: 1.5865 - val_categorical_accuracy: 0.4088
Epoch 4/5
313/313 [==============================] - 438s 1s/step - loss: 1.9565 - categorical_accuracy: 0.2893 - val_loss: 1.4933 - val_categorical_accuracy: 0.4464
Epoch 5/5
63/63 [==============================] - 57s 900ms/step - loss: 1.4421 - categorical_accuracy: 0.4550
True
test accuracy:0.45500001311302185



categorical_accuracy,▁▃▄▆█
epoch,▁▃▅▆█
loss,█▅▃▂▁
test accuracy,▁
val_categorical_accuracy,▁▄▅▇█
val_loss,█▆▄▃▁
best_epoch,4
best_val_loss,1.37347
categorical_accuracy,0.32293
epoch,4
loss,1.81315


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1omo9inq with config:
wandb: 	augmentation: False
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	drop_out: 0.4
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	models: inception_v2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
313/313 [==============================] - 477s 1s/step - loss: 1.3611 - categorical_accuracy: 0.6723 - val_loss: 0.5645 - val_categorical_accuracy: 0.8234
Epoch 2/10
313/313 [==============================] - 463s 1s/step - loss: 0.9679 - categorical_accuracy: 0.7502 - val_loss: 0.4303 - val_categorical_accuracy: 0.8599
Epoch 3/10
313/313 [==============================] - 476s 2s/step - loss: 0.7711 - categorical_accuracy: 0.7934 - val_loss: 0.3350 - val_categorical_accuracy: 0.8880
Epoch 4/10
313/313 [==============================] - 388s 1s/step - loss: 0.6219 - categorical_accuracy: 0.8232 - val_loss: 0.2305 - val_categorical_accuracy: 0.9209
Epoch 5/10
313/313 [==============================] - 383s 1s/step - loss: 0.5026 - categorical_accuracy: 0.8575 - val_loss: 0.1920 - val_categorical_accuracy: 0.9320
Epoch 6/10
313/313 [================